In [1]:
!pip install fastapi uvicorn nest-asyncio pyngrok panns-inference joblib librosa python-multipart torch xgboost==2.1.4

In [2]:
import xgboost
print(xgboost.__version__)

2.1.4


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [3]:
!ngrok config add-authtoken 2xuv9CBvzNoX4piS1NFz0GJHEDX_5Z7r6fmd8kERUv4hb3NrH

Authtoken saved to configuration file: /home/ec2-user/.config/ngrok/ngrok.yml


In [4]:
from fastapi import FastAPI, Request
from pydantic import BaseModel
import base64
import numpy as np
import librosa
import io
import torch
import joblib
from panns_inference import AudioTagging
from fastapi.responses import JSONResponse

# Load models
xgb_model = joblib.load("whale_xgb_model.pkl")
class_map = joblib.load("class_map.pkl")
idx_to_class = {v: k for k, v in class_map.items()}
at_model = AudioTagging(checkpoint_path=None, device="cpu")

print("✅ Models loaded")

Checkpoint path: /home/ec2-user/panns_data/Cnn14_mAP=0.431.pth
Using CPU.
✅ Models loaded


In [5]:
app = FastAPI()

class AudioPayload(BaseModel):
    audio_base64: str

@app.post("/predict")
async def predict(payload: AudioPayload):
    try:
        audio_bytes = base64.b64decode(payload.audio_base64)
        waveform, _ = librosa.load(io.BytesIO(audio_bytes), sr=32000, mono=True)
        if len(waveform) < 32000:
            waveform = np.pad(waveform, (0, 32000 - len(waveform)), mode='constant')
        waveform = waveform[None, :]
        _, embedding = at_model.inference(waveform)
        proba = xgb_model.predict_proba(embedding)[0]
        prediction = idx_to_class[int(np.argmax(proba))]

        return JSONResponse({
            "prediction": prediction,
            "confidence": float(np.max(proba)),
            "top_predictions": {
                idx_to_class[i]: float(prob) for i, prob in enumerate(proba)
            }
        })
    except Exception as e:
        return JSONResponse(status_code=500, content={"error": str(e)})

In [6]:
from pyngrok import ngrok

public_url = ngrok.connect(8000)
print("🌐 Public URL:", public_url)


🌐 Public URL: NgrokTunnel: "https://b5ad-34-236-202-207.ngrok-free.app" -> "http://localhost:8000"


In [ ]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()

uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [6038]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     142.68.132.38:0 - "POST /predict HTTP/1.1" 200 OK
